# Lab 3: Determination of Protein Concentration

In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns 
import matplotlib as mp
import matplotlib.pyplot as plt 

from scipy import stats
from textwrap import wrap

%matplotlib inline
%config InlineBackend.figure_format = 'pdf'

In [2]:
BR = pd.read_csv('/Users/kev/Downloads/BCMB 301A Lab Techniques/Lab 3 - Determination of Protein Concentrations/Data/Biuret.csv', 
                 header=1)

LR = pd.read_csv('/Users/kev/Downloads/BCMB 301A Lab Techniques/Lab 3 - Determination of Protein Concentrations/Data/Lowry.csv', 
                 header=1)

BF = pd.read_csv('/Users/kev/Downloads/BCMB 301A Lab Techniques/Lab 3 - Determination of Protein Concentrations/Data/Bradford.csv', 
                 header=1)

UN = pd.read_csv('/Users/kev/Downloads/BCMB 301A Lab Techniques/Lab 3 - Determination of Protein Concentrations/Data/Unknown.csv', 
                 header=1)

UV = pd.read_csv('/Users/kev/Downloads/BCMB 301A Lab Techniques/Lab 3 - Determination of Protein Concentrations/Data/Ultraviolet.csv', 
                 header=0)

In [3]:
# Get average absorbance
BR_ave = pd.DataFrame(BR.iloc[:,1:].mean(axis=1))
LR_ave = pd.DataFrame(LR.iloc[:,1:].mean(axis=1))
BF_ave = pd.DataFrame(BF.iloc[:,1:].mean(axis=1))
UN_ave = pd.DataFrame(UN.iloc[:,1:].mean(axis=1))
BR_ave.columns = ['Average']
LR_ave.columns = ['Average']
BF_ave.columns = ['Average']
UN_ave.columns = ['Average']
BR = BR.join(BR_ave)
LR = LR.join(LR_ave)
BF = BF.join(BF_ave)
UN = UN.join(UN_ave)

In [4]:
# Get mg of BSA 
BR_g = pd.DataFrame(BR['BSA (10g/l) [ml]'] * 10)
LR_g = pd.DataFrame(LR['BSA (100 mg/l) [ml]'] * 0.1)
BF_g = pd.DataFrame(BF['BSA (100 mg/l) [ul]'] * 10**(-4))
BR_g.columns = ['BSA [mg]']
LR_g.columns = ['BSA [mg]']
BF_g.columns = ['BSA [mg]']
BR = BR.join(BR_g)
LR = LR.join(LR_g)
BF = BF.join(BF_g)

In [5]:
BR

,BSA (10g/l) [ml],Set 1,Set 2,Average,BSA [mg]
0,0.0,0.000,0.000,0.0000,0.0
1,0.2,0.045,0.079,0.0620,2.0
2,0.4,0.109,0.160,0.1345,4.0
3,0.6,0.194,NaN,0.1940,6.0
4,0.8,NaN,0.337,0.3370,8.0
5,1.0,0.361,0.372,0.3665,10.0


In [6]:
LR

,BSA (100 mg/l) [ml],Set 1,Set 2,Average,BSA [mg]
0,0.0,0.000,0.000,0.0000,0.00
1,0.2,0.133,0.118,0.1255,0.02
2,0.4,0.231,0.237,0.2340,0.04
3,0.6,0.347,0.329,0.3380,0.06
4,0.8,0.389,0.444,0.4165,0.08
5,1.0,0.520,0.523,0.5215,0.10


In [7]:
BF

,BSA (100 mg/l) [ul],Set 1,Set 2,Average,BSA [mg]
0,0,-0.005,0.005,0.0000,0.000
1,20,0.081,0.098,0.0895,0.002
2,40,0.144,0.146,0.1450,0.004
3,60,0.166,0.160,0.1630,0.006
4,80,0.205,0.198,0.2015,0.008
5,100,0.270,0.271,0.2705,0.010


## F. Practical Assessment

In [23]:
def plot_abs(Dat, X, Y, Abs, Yerr, ploterror=False):
    slope, intercept, r_value, p_value, std_err = stats.linregress(Dat[X], Dat[Y])

    plt.figure()
    sns.regplot(x = X, y=Y, data=Dat, 
                scatter_kws={'s':8}, 
                line_kws={'linewidth':2, 'label':r"A={0:.3g}M+({1:.3g}); R$^2$={2:.3g}".format(slope, intercept, r_value**2)})
    
    if ploterror == True: 
        plt.errorbar(x=Dat[X], y=Dat[Y], yerr=Yerr, scatterpoints=None, linewidth=0, elinewidth=1, capsize=2,
                    label='Uncertainty: {0:.4f}'.format(Yerr))
    
#     F_title = r'Changes in average absorbance at {0} nm ($\bar{{A}}_{1}$) with respect to the quantity of BSA, with a 95% confidence interval'.format(Abs, {Abs})
#     plt.title("\n".join(wrap(F_title, 80)))
    plt.ylabel(r'$\bar{{A}}_{0}$'.format({Abs}))
    plt.legend(loc=2)
    plt.grid()
    plt.show()
    return slope, intercept

In [22]:
BR_s, BR_i = plot_abs(Dat=BR, X='BSA [mg]', Y='Average', Abs=550, Yerr=(2*0.0005**2)**0.5)
LR_s, LR_i = plot_abs(Dat=LR, X='BSA [mg]', Y='Average', Abs=750, Yerr=(2*0.0005**2)**0.5)
BF_s, BF_i = plot_abs(Dat=BF, X='BSA [mg]', Y='Average', Abs=595, Yerr=(2*0.0005**2)**0.5)

<Figure size 432x288 with 1 Axes>

<Figure size 432x288 with 1 Axes>

<Figure size 432x288 with 1 Axes>

## Figure out unknown BSA concentration

In [10]:
Slope = np.array([BR_s, LR_s, BF_s])
Inter = np.array([BR_i, LR_i, BF_i])
MG = (UN.loc[:,'Average'] - Inter.T)/Slope.T # Protein mass in mg
MG = np.array(MG)
VL = np.array([1, 1, 0.1]) # Volume of sample
DF = np.array([1, 100, 100]) # Dilution factor 
EC = MG*DF/VL # Experimental concentration 
AC = np.array([8, 8, 8]) # Actual concentration 
PE = np.abs(EC-AC)/AC * 100 # Percentage Error 

AR = np.array([Slope, Inter, MG, VL, DF, EC, AC, PE]).T
AR = pd.DataFrame(AR)

AR.columns = ['Slope', 'Intercept', 'Mass (mg)', 'Volume (ml)', 'Dilution', 'Exp. (g/l)', 'Act. (g/l)', 'Percent Error']
UN = UN.join(AR)

In [11]:
UN.T

,0,1,2
Assay,Biuret 550nm,Lowry 750nm,Bradford 595nm
Set 1,0.366,0.489,0.231
Set 2,0.39,0.488,0.238
Average,0.378,0.4885,0.2345
Slope,0.0388143,5.12071,24.3786
Intercept,-0.0117381,0.0165476,0.0230238
Mass (mg),10.0411,0.0921653,0.00867468
Volume (ml),1,1,0.1
Dilution,1,100,100
Exp. (g/l),10.0411,9.21653,8.67468


## Beer's Law

In [12]:
UV_bsa_ave = pd.DataFrame(UV.loc[:,'BSA 1':'BSA 2'].mean(axis=1))
UV_unk_ave = pd.DataFrame(UV.loc[:,'Set 1':'Set 2'].mean(axis=1))
UV_bsa_ave.columns = ['BSA Average']
UV_unk_ave.columns = ['Average']
ER = UV_bsa_ave.join(UV_unk_ave)

UV_cuv_len = 1 
UV_DF = 8 
UV_conc = 10 / UV_DF

CR = np.array([[UV_cuv_len], [UV_DF], [UV_conc]])
CR = pd.DataFrame(CR.T)
CR.columns = ['Cuvette length (cm)', 'Dilution', 'Concentration (Diluted)']
UV = UV.join([ER, CR])

a_val = pd.DataFrame(UV.loc[:,'BSA Average']/(UV.loc[:,'Cuvette length (cm)'] * UV.loc[:,'Concentration (Diluted)']))
a_val.columns = [r'a ((M$\cdot$cm$)^{-1}$)']
UV = UV.join(a_val)

In [13]:
UV_unk_conc = np.array(UV.loc[:,'Average'] * UV_DF / (UV.loc[:,'Cuvette length (cm)'] * UV.loc[:,r'a ((M$\cdot$cm$)^{-1}$)']))
UV_act_conc = np.array([8])
UV_pc_err = np.abs(UV_unk_conc-UV_act_conc)/UV_act_conc * 100
DR = np.array([UV_unk_conc, UV_act_conc, UV_pc_err]).T
DR = pd.DataFrame(DR)
DR.columns = ['Exp. (g/l)', 'Act. (g/l)', 'Percent error']
UV = UV.join(DR)

In [14]:
UV.T

,0
Assay,UV 280nm
BSA 1,0.558
BSA 2,0.593
Set 1,0.547
Set 2,0.576
BSA Average,0.5755
Average,0.5615
Cuvette length (cm),1
Dilution,8
Concentration (Diluted),1.25


## Percentage Difference 

In [15]:
P1 = UN.loc[:,'Assay':'Average']
P2 = UV.loc[:,['Assay','Set 1','Set 2','Average']]

In [16]:
PD = pd.concat([P1, P2], sort=False).reset_index(drop=True)

In [17]:
DF = pd.DataFrame(np.abs(PD.loc[:,'Set 1']-PD.loc[:,'Set 2']))
DF.columns = ['Abs Diff']
pd = pd.DataFrame(DF.loc[:,'Abs Diff']/PD.loc[:,'Average'] * 100).reset_index(drop=True)
pd.columns = ['Percent Diff']

P3 = PD.join([DF, pd])

In [18]:
P3

,Assay,Set 1,Set 2,Average,Abs Diff,Percent Diff
0,Biuret 550nm,0.366,0.390,0.3780,0.024,6.349206
1,Lowry 750nm,0.489,0.488,0.4885,0.001,0.204708
2,Bradford 595nm,0.231,0.238,0.2345,0.007,2.985075
3,UV 280nm,0.547,0.576,0.5615,0.029,5.164737
